# Exercise 1: Elastic rotating bar

In [1]:
from ngsolve import *
from netgen.occ import *
import ipywidgets as widgets
from ngsolve.comp import GlobalSpace
from ngsolve.webgui import Draw
from ngsolve.solvers import Newton
from ngsolve.comp import DifferentialSymbol
import numpy as np
# ngsglobals.msg_level=10
import time

In [2]:
def curl(gf: GridFunction)->GridFunction:
    return CoefficientFunction( (0, 0, grad(gf)[1,0]-grad(gf)[0,1]) )

In [3]:
shape = Rectangle(1, 0.1).Face()

mesh = Mesh(OCCGeometry(shape, dim=2).GenerateMesh(maxh=0.02))

In [4]:
# density
rho = 1

mass = rho*Integrate(CF(1)*dx, mesh)

# move shape's center of mass to origin; set start translation accordingly
center_of_mass = (1/mass*Integrate(rho*x*dx, mesh), 1/mass*Integrate(rho*y*dx, mesh))

shape = shape.Move((-center_of_mass[0], -center_of_mass[1], 0))
# shape += Circle((0, 0), 0.1).Face()
mesh = Mesh(OCCGeometry(shape, dim=2).GenerateMesh(maxh=0.02))

# moment_of_inertia = Integrate((x**2 + y**2)*dx, mesh)

r = list(center_of_mass)
r = [0, 0]


In [5]:
Draw(mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

BaseWebGuiScene

## time-stepping

Variables are

* position $\phi = r + R x$, an affine linear function, with constraint $R^T R = I$, i.e. $R$ is orthogonal (and thus, by continuity, a rotation matrix)
* velocity $v = a + b \wedge x$, in body-frame

In [6]:
E, nu = 100, 0.2
mu  = E / 2 / (1+nu)
lam = E * nu / ((1+nu)*(1-2*nu))

def Stress(strain):
    return 2*mu*strain + lam*Trace(strain)*Id(2)

In [7]:
tau = 0.00001
tend = 0.5
u0 = CF((0, 0))
v0 =  50*CF((y, -x)) # CF((0, (y+100)))

fes = VectorH1(mesh, order=3)
u,testf = fes.TnT()
aform = InnerProduct(Stress(Sym(Grad(u))), Sym(Grad(testf)))*dx

a = BilinearForm(aform).Assemble()
mstar = BilinearForm(rho*u*testf*dx + tau**2/4*aform).Assemble()
mstarinv = mstar.mat.Inverse(inverse="sparsecholesky")
f = LinearForm(fes).Assemble()

In [8]:
gfu = GridFunction(fes)
gfv = GridFunction(fes)

gfu.Set(u0)
# gfv.Set(0.001*CF(((y - shape.center[1]), -(x - shape.center[0]))))
gfv.Set(v0)

gf_graphical = GridFunction(fes)
scene = Draw (gf_graphical, deformation=True, order=3)

# pseudo-Newmark (sloppy force calculation)
for j in range(int(tend/tau)):
    # print(Cross(curl(gfv), Cross(curl(gfv), CF((x, y, z))))[1:2]*testf)
    f = LinearForm(rho*Cross(curl(gfv), Cross(curl(gfv), CF((x, y, z))))[0:2]*testf*dx).Assemble()
    
    gfu.vec.data += tau/2 * gfv.vec
    gfv.vec.data += tau * mstarinv * (f.vec - a.mat * gfu.vec)
    gfu.vec.data += tau/2 * gfv.vec

    # fit in rigid body translation
    # center_of_mass = (1/mass*Integrate(rho*gfu[0]*Norm(Det(Grad(gfu)))*dx, mesh), 1/mass*Integrate(rho*gfu[1]*Norm(Det(Grad(gfu)))*dx, mesh))
    # r[0] += center_of_mass[0]
    # r[1] += center_of_mass[1]
    # gfu.Set(gfu - CF(center_of_mass))

    gf_graphical.Set(gfu) # CF((r[0], r[1])) + 
    # print(center_of_mass)

    scene.Redraw()
    time.sleep(0.1)


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

/home/norberth/venveria/venvs/sc/lib/python3.13/site-packages/netgen/webgui.py:20: RuntimeWarning: overflow encountered in cast
  values = np.array(data.flatten(), dtype=dtype)


KeyboardInterrupt: 

In [ ]:
help(Circle)